In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
#download cleaned cycle hire data
cycle_df = pd.read_csv('cycle_df_weather.csv', parse_dates=[0, 1, 3])

In [3]:
features_df = cycle_df.drop(columns=['Start Date', 'End Date', 'StartStation Name', 'EndStation Name',
                                    'EndStation Id', 'hour'])

aggfunc = {'Duration(mins)' : 'mean', 'Day' : 'max', 'day_code' : 'max', 'Temperature' : 'mean', 'Wind Speed' : 'mean',
           'w_cond_Bad weather' : 'mean', 'w_cond_Good weather' : 'mean', 'w_cond_OK weather' : 'mean', 
           'w_cond_Very bad weather' : 'mean', 'Bike Id': 'count'}
features_df = features_df.groupby(['Date', 'StartStation Id']).agg(aggfunc).rename(columns={'Bike Id' : 'Count'})
features_df['day_code'] = features_df['day_code'].replace({'PH' : 'WD'})

In [4]:
day_bf_count = []
for idx, data in features_df.iterrows():
    
    date, station_id = idx
    date_bf = date - timedelta(days=1)
    
    if date == features_df.iloc[0].name[0]:
        count = np.nan      
    else:
        try:
            count = features_df.loc[(date_bf, station_id)]['Count']
        except:
            count = 0 
    
    day_bf_count.append(count)

In [5]:
features_df['day_bf_count'] = day_bf_count

In [7]:
features_df.reset_index()
#avg duration last 7 days or day before
#day-code replace with binary values
#temperature in real model will be forecast
#start at station count + end at station count (day before)
#split features and Y data

,Date,StartStation Id,Duration(mins),Day,day_code,Temperature,Wind Speed,w_cond_Bad weather,w_cond_Good weather,w_cond_OK weather,w_cond_Very bad weather,Count,day_bf_count
0,2019-06-01,1,34.062500,5,W,21.812500,15.562500,0.0,1.0,0.0,0.0,16,NaN
1,2019-06-01,2,37.562500,5,W,22.112500,13.412500,0.0,1.0,0.0,0.0,80,NaN
2,2019-06-01,3,42.894737,5,W,20.526316,10.894737,0.0,1.0,0.0,0.0,19,NaN
3,2019-06-01,4,26.818182,5,W,20.000000,11.045455,0.0,1.0,0.0,0.0,22,NaN
4,2019-06-01,5,25.378378,5,W,20.621622,13.351351,0.0,1.0,0.0,0.0,37,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68000,2019-08-27,834,25.268293,1,W,21.609756,7.853659,0.0,1.0,0.0,0.0,41,60.0
68001,2019-08-27,835,15.848485,1,W,27.393939,17.121212,0.0,1.0,0.0,0.0,33,17.0
68002,2019-08-27,836,20.363636,1,W,22.818182,8.393939,0.0,1.0,0.0,0.0,33,20.0
68003,2019-08-27,838,14.454545,1,W,25.363636,13.454545,0.0,1.0,0.0,0.0,44,7.0
